In [31]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import json
import sqlite3
import numpy as np
%matplotlib inline

thenums_df = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz')
thenums_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0



## I made a function that turned object columns into integer colums ###

I ran this function onto the columns of this dataframe


In [32]:
# This function turns series columns into integers
def make_column_integer(df, series):
    df[series] = df[series].str.replace('$', '')
    df[series] = df[series].str.replace(',', '')
    df[series] = df[series].astype(int)

# Turns all the columns in this dataframe into integers
make_column_integer(thenums_df, 'production_budget')
make_column_integer(thenums_df, 'domestic_gross')
make_column_integer(thenums_df, 'worldwide_gross')


## I then made 2 new columns: 

One column called "total_gross" which was the sum of the domestic and worldwide grossings, and another column called "total_profit" that is the total gross minus the production cost.



In [33]:
thenums_df['total_gross']= (thenums_df['domestic_gross'])+(thenums_df['worldwide_gross'])
thenums_df['total_profit'] = (thenums_df['total_gross']-thenums_df['production_budget'])
thenums_df.sort_values('total_profit', ascending = False, inplace=True)
thenums_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,total_gross,total_profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,3536852904,3111852904
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2989973445,2683973445
42,43,"Dec 19, 1997",Titanic,200000000,659363944,2208208395,2867572339,2667572339
6,7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,2726949682,2426949682
33,34,"Jun 12, 2015",Jurassic World,215000000,652270625,1648854864,2301125489,2086125489
...,...,...,...,...,...,...,...,...
480,81,"Dec 31, 2019",Army of the Dead,90000000,0,0,0,-90000000
479,80,"Dec 13, 2017",Bright,90000000,0,0,0,-90000000
341,42,"Jun 14, 2019",Men in Black: International,110000000,3100000,3100000,6200000,-103800000
194,95,"Dec 31, 2020",Moonfall,150000000,0,0,0,-150000000


## I imported a new dataframe from the IMDB database that contained the genres of the movies.

In [28]:
imdb_title_basics = pd.read_csv('data/zippedData/imdb.title.basics.csv.gz')

In [29]:
imdb_title_basics

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,NaN


In [34]:
genres_and_profits = pd.merge(imdb_title_basics, thenums_df, how='inner', left_on='primary_title', right_on='movie')

In [37]:
genres_and_profits.sort_values('total_profit', ascending=False)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,total_gross,total_profit
1747,tt1775309,Avatar,Abatâ,2011,93.0,Horror,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,3536852904,3111852904
2776,tt8852130,Titanic,Titanic,2018,115.0,Family,43,"Dec 19, 1997",Titanic,200000000,659363944,2208208395,2867572339,2667572339
2775,tt2495766,Titanic,Titanic,2012,NaN,Adventure,43,"Dec 19, 1997",Titanic,200000000,659363944,2208208395,2867572339,2667572339
3360,tt4154756,Avengers: Infinity War,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,2726949682,2426949682
12,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",34,"Jun 12, 2015",Jurassic World,215000000,652270625,1648854864,2301125489,2086125489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3740,tt7504726,Call of the Wild,Call of the Wild,2020,NaN,"Adventure,Animation,Family",36,"Feb 21, 2020",Call of the Wild,82000000,0,0,0,-82000000
792,tt1305591,Mars Needs Moms,Mars Needs Moms,2011,88.0,"Adventure,Animation,Family",94,"Mar 11, 2011",Mars Needs Moms,150000000,21392758,39549758,60942516,-89057484
3566,tt5519340,Bright,Bright,2017,117.0,"Action,Crime,Fantasy",80,"Dec 13, 2017",Bright,90000000,0,0,0,-90000000
2580,tt2283336,Men in Black: International,Men in Black: International,2019,115.0,"Action,Adventure,Comedy",42,"Jun 14, 2019",Men in Black: International,110000000,3100000,3100000,6200000,-103800000
